In [1]:
import pandas as pd
import numpy as np
import folium
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import requests

## <u>Week 3 project - Phase 1:</u>

#### We read and import the data from wikipedia

In [2]:
url = r"https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
# send get request to read the wiki page
web_text = requests.get(url).text

# we parse imported text via lxml parser
soup = BeautifulSoup(web_text, 'lxml')

#### After getting the text of the page we can read the table now

In [4]:
# we create empty lists, one per each column to hold our data
postal_code, borough, neighbourhood = [], [], []

# we loop thorugh the tables nad record the data of each row
for items in soup.find('table', class_='wikitable').find_all('tr')[1::1]:
    data = items.find_all(['th','td'])
    try:
        # we tray recording them inside each list
        postal_code.append(data[0].text)
        borough.append(data[1].text)
        neighbourhood.append(data[2].text)
    except IndexError:pass

#### Then we create data frame of the collected data from the table

In [5]:
# we create a dcitionary on the flyt to convert it into a dataframe with header
df_main = pd.DataFrame({"postal_code":postal_code, "borough":borough, "neighbourhood":neighbourhood})

In [6]:
df_main.head()

,postal_code,borough,neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


#### We clean the data from any unwanted strings

In [7]:
# as we see strange this "\n" char, we should know how many rows have the same issue 
df_main.neighbourhood.str.contains("\n").count()

287

In [8]:
# then we test the replacement before applying it
df_main.neighbourhood.str.replace("\n","").head()

0        Not assigned
1        Not assigned
2           Parkwoods
3    Victoria Village
4        Harbourfront
Name: neighbourhood, dtype: object

In [9]:
# we applt the replacment and check our data for confirmation
df_main.neighbourhood = df_main.neighbourhood.str.replace("\n","")

In [10]:
df_main.head()

,postal_code,borough,neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Then we delete all not assgined borough

In [11]:
# we drope the unnecessary rows as required per the project guide
df_main.drop(df_main.loc[df_main["borough"]=="Not assigned"].index,axis=0, inplace=True)

In [12]:
# we check that deleteing those rows is made successfully
df_main.head()

,postal_code,borough,neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


#### After that, We should assign the borough to any not assigned neigbourhood

In [13]:
# we assign the empty neigborhood the same name of its borough as required by the project guide
df_main.loc[df_main["neighbourhood"]=="Not assigned",['neighbourhood']] = df_main.borough

#### We check duplicated rows

In [14]:
df_main[df_main.postal_code.duplicated()].count()

postal_code      107
borough          107
neighbourhood    107
dtype: int64

#### We copy the main datafram into a transtional data fram so we can transform it without touching our main data frame

In [59]:
# we copy our data frame into another one
df_trans = df_main.copy()

In [61]:
# we merge the postal code and bouroug into one line to determine the unique values based on both of them
df_trans["postal_code"] = df_main["postal_code"] + "|" + df_main["borough"] 

In [62]:
df_trans.head()

,postal_code,borough,neighbourhood
2,M3A|North York,North York,Parkwoods
3,M4A|North York,North York,Victoria Village
4,M5A|Downtown Toronto,Downtown Toronto,Harbourfront
5,M6A|North York,North York,Lawrence Heights
6,M6A|North York,North York,Lawrence Manor


#### After we merged the columns to make sure that the dataframe is ready for our merge process
#### we wrote a simple funciton that gets the unique values of a column and merges the other column based on the duplicates

In [26]:
def merge_dubplicates(data, dup_col, merge_col, merge_col_index):
    """This function merges the duplicates of another column based on a column and returns a dataframe merged and cleaned from duplicates"""
    row = []
    # we loop through unique values in the defined column
    for i in data[dup_col].unique() :
        ne = ''
        # we get the rows associated with this value
        for j in data.loc[data[dup_col]==i].iterrows():
            # we merge the value of the other columns
            ne =   j[1][merge_col_index] + ", " + ne 
        # append both of the columns to one list
        row.append([i, ne[0:-2]])
    # finally we create a dataframe from the list with the same names of the columns
    df = pd.DataFrame(row, columns=[dup_col, merge_col])
    return df

In [63]:
# we apply our function and store the new data frame a variable
df_merged = merge_dubplicates(df_trans, "postal_code", "neighbourhood", 2)

#### After we got the merged data we split it again into the final shape

In [71]:
# we split the merged columns 
df_merged[["postal_code","borough"]] = df_merged["postal_code"].str.split("|", expand=True)

In [75]:
#we arrange the columns again
df_merged = df_merged[["postal_code","borough","neighbourhood"]]

In [76]:
df_merged.head()

,postal_code,borough,neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,Queen's Park


#### We save it for further phases and check the shape as required for this point of the project

In [78]:
# we save the final data frame in CSV file
df_merged.to_csv("final_dataset.csv")

In [79]:
df_merged.shape

(103, 3)